In [ ]:
import os
import sys
import gzip
import json
import time
import urllib
import tempfile
import collections

import numpy as np
import pandas as pd
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

import pybiomart
from gene_map import GeneMapper
from tqdm import tqdm_notebook as tqdm

from utils import load_config, split_df_row
from tad_helper_functions import parse_tad_annotations

In [ ]:
from tqdm import tqdm as tqdm_orig
tqdm_orig.pandas()

pd.set_option('display.max_columns', 99)

In [ ]:
sns.set_context('talk')

In [ ]:
config = load_config()

cache_dir = config['output_dirs']['cache']
images_dir = config['output_dirs']['images']

# Load data

In [ ]:
df_disgenet = pd.read_table(
    config['input_files']['raw_disgenet'],
    usecols=['snpId','diseaseId','diseaseName','source'])
df_disgenet['diseaseIdType'] = 'UMLS_CUI'

In [ ]:
df_disgenet.head()

In [ ]:
results_dir = config['output_dirs']['results']
tad_data_fname = f'{results_dir}/tads_hg38.tsv'

# Integrate latest GWAS-catalog version

Column definitions: https://www.ebi.ac.uk/gwas/docs/methods/curation

In [ ]:
df_gwascat = pd.read_table(config['input_files']['raw_gwascatalog'], low_memory=False)

#df_gwascat = df_gwascat[['SNP_ID_CURRENT', 'MAPPED_TRAIT_URI', 'MAPPED_TRAIT']]
df_gwascat.dropna(subset=['SNP_ID_CURRENT', 'MAPPED_TRAIT_URI', 'MAPPED_TRAIT'], inplace=True)
df_gwascat.rename(columns={
    'SNP_ID_CURRENT': 'snpId', 'MAPPED_TRAIT_URI': 'diseaseId', 'MAPPED_TRAIT': 'diseaseName'
}, inplace=True)

df_gwascat['snpId'] = df_gwascat['snpId'].apply(lambda x: f'rs{x}')
df_gwascat['source'] = 'GWASCUSTOM'
df_gwascat = split_df_row(df_gwascat, 'diseaseId', ',')
df_gwascat['diseaseId'] = df_gwascat['diseaseId'].apply(lambda x: x.split('/')[-1])
df_gwascat['diseaseIdType'] = df_gwascat['diseaseId'].apply(lambda x: x.split('_')[0])

# convert BETA to odds ratio
df_gwascat['odds_ratio'] = df_gwascat['OR or BETA'].apply(lambda x: np.exp(x) if x < 1 else x)
if config['filters']['OR_threshold'] is not None:
    df_gwascat = df_gwascat[df_gwascat['odds_ratio'] > config['filters']['OR_threshold']]

df_gwascat.sample(5)

In [ ]:
df = pd.concat([df_gwascat])  #df_disgenet, 
df.sample(5)

## Infer associated gene(s)

Possible columns:
* REPORTED GENE(S): gene reported by author
* MAPPED GENE: Gene(s) mapped to the strongest SNP (if SNP is intergenic uses upstream and downstream genes)
* SNP_GENE_IDS: Entrez Gene ID

In [ ]:
df[['REPORTED GENE(S)', 'MAPPED_GENE', 'SNP_GENE_IDS']].sample(5)

In [ ]:
gene_source = config['parameters']['associated_gene_source']

if gene_source == 'reported':
    # are gene names, must be mapped to ENTREZ
    raw_genes = df['REPORTED GENE(S)'].str.split(', ').tolist()
    
    gene_blacklist = {'intergenic', 'NR'}
    cur_genes = [g for gs in raw_genes if not isinstance(gs, float) for g in gs if g not in gene_blacklist]  # isinstance(gs,float) -> gs==np.nan
    
    gm = GeneMapper()
    df_map = gm.query(id_list=cur_genes, source_id_type='Gene_Name', target_id_type='GeneID')
    name2id = df_map.set_index('ID_from').to_dict()['ID_to']
    
    entrez_genes = [None 
                    if isinstance(gs, float) 
                    else [name2id[g] for g in gs if g in name2id]
                    for gs in raw_genes]
elif gene_source == 'mapped':
    # are already ENTREZ IDs
    raw_genes = df['SNP_GENE_IDS'].str.split(', ').tolist()
    entrez_genes = [None if isinstance(gs, float) else gs for gs in raw_genes]
else:
    raise RuntimeError(f'Invalid gene source: "{gene_source}"')

In [ ]:
df['associated_genes'] = [None if gs is None else ','.join(gs) for gs in entrez_genes]
df[['REPORTED GENE(S)', 'MAPPED_GENE', 'SNP_GENE_IDS', 'associated_genes']].sample(5)

# Parse Ontology OWLs

In [ ]:
def parse_owl_file(soup, relevant_terms):
    """ Extract requested terms from OWL-file
    """
    node_owl_data = {}
    for entry in tqdm(soup.find_all('Class')):
        doid = entry['rdf:about'].split('/')[-1]

        # get label
        lbl = entry.find('rdfs:label').get_text()

        # get requested terms
        term_map = {term: [] for term in relevant_terms}
        for xref in entry.find_all('oboInOwl:hasDbXref'):
            txt = xref.get_text()
            for term in relevant_terms:
                if txt.startswith(f'{term}:'):
                    idx = txt.split(':')[-1]
                    term_map[term].append(idx)

        assert doid not in node_owl_data, doid
        node_owl_data[doid] = {
            'label': lbl,
            'terms': term_map
        }
        
    return node_owl_data

## DOID

In [ ]:
with open(config['input_files']['disease_ontology']) as fd:
    soup_doid = BeautifulSoup(fd, 'xml')

In [ ]:
node_owl_data_doid = parse_owl_file(soup_doid, ['UMLS_CUI'])

## Save EFO disease labels

In [ ]:
with open(config['input_files']['exp_factor_ontology']) as fd:
    soup = BeautifulSoup(fd, 'xml')

In [ ]:
efo_lbl_map = {}
for entry in tqdm(soup.find_all('Class')):
    if not entry.has_attr('rdf:about'):
        continue
    
    efo = entry['rdf:about'].split('/')[-1]
    lbl = entry.find('rdfs:label').get_text()

    assert efo not in efo_lbl_map
    efo_lbl_map[efo] = lbl

In [ ]:
results_dir = config['output_dirs']['results']

df_efolabels = pd.DataFrame(list(efo_lbl_map.items()), columns=['EFO', 'label'])
df_efolabels.to_csv(f'{results_dir}/disease_efolabels.csv', index=False)

print(df_efolabels.shape)
df_efolabels.sample(5)

# Disease ontology as tree

## Load data

In [ ]:
def load_ontology_network(fname_in):
    type_ = os.path.basename(fname_in).split('.')[0]
    #assert type_ in ('efo', 'doid'), f'Invalid type: {type_}'
    
    fname_out = f'{cache_dir}/{type_}_graph.edgelist.gz'
    if not os.path.exists(fname_out):
        import onto2nx  # https://github.com/cthoyt/onto2nx
        nx.write_edgelist(onto2nx.parse_owl_rdf(fname_in), fname_out)
    else:
        print('Cached', fname_out)
        
    graph = nx.read_edgelist(fname_out, create_using=nx.DiGraph()).reverse()
    graph.name = type_
    return graph

In [ ]:
doid_graph = load_ontology_network(config['input_files']['disease_ontology'])
print(nx.info(doid_graph))

In [ ]:
efo_graph = load_ontology_network(config['input_files']['exp_factor_ontology'])
print(nx.info(efo_graph))

## Map UMLS_CUI to DOID node

In [ ]:
doid_umls_map = {}
for node, data in tqdm(node_owl_data_doid.items()):
    assert set(data['terms'].keys()) == set(['UMLS_CUI'])
    doid_umls_map[node] = data['terms']['UMLS_CUI']

## Find cancer subtree

In [ ]:
given_diseases = set(df.diseaseId.unique())

# gather all possible disease-nodes
all_nodes_umls = [umls for doid in doid_graph.nodes() for umls in doid_umls_map[doid]]
all_nodes_efo = list(nx.descendants(efo_graph, 'EFO_0000408')) + ['EFO_0000408']  # disease subtree (vs traits, ...)

all_nodes = set(all_nodes_efo + all_nodes_umls) & given_diseases

# extract all cancer diseases
cancer_nodes_doid = (list(nx.descendants(doid_graph, 'DOID_162')) + ['DOID_162'])  # cancer subtree
cancer_nodes_umls = [umls for doid in cancer_nodes_doid for umls in doid_umls_map[doid]]

cancer_nodes_efo = list(nx.descendants(efo_graph, 'EFO_0000311')) + ['EFO_0000311']  # cancer subtree

cancer_nodes = set(cancer_nodes_efo + cancer_nodes_umls) & given_diseases
assert cancer_nodes <= all_nodes
print(f'#various nodes: {len(cancer_nodes)}/{len(all_nodes)}/{len(given_diseases)}')

In [ ]:
# do cancer-classification
data_cancer = []
for disease in tqdm(df['diseaseId'].unique()):
    if disease in all_nodes:
        data_cancer.append((disease, disease in cancer_nodes))
    
df_iscancer = pd.DataFrame(data_cancer, columns=['diseaseId','is_cancer'])
df_iscancer.sample(5)

In [ ]:
results_dir = config['output_dirs']['results']
df_iscancer.to_csv(f'{results_dir}/disease_cancer_classification.csv', index=False)

# SNP annotations

## Retrieve VEP annotations

Variant consequence ontology: http://www.sequenceontology.org/browser/current_release

In [ ]:
snps = df['snpId'].unique().tolist()

In [ ]:
coord_version = config['parameters']['source_genomiccoordinates_version']

if coord_version == 'hg38':
    mart_host = 'www.ensembl.org'
elif coord_version == 'hg19':
    mart_host = 'grch37.ensembl.org'
else:
    raise RuntimeError(f'Invalid coordinate version: "{coord_version}"')
    
print(f'Using {mart_host}')

In [ ]:
%%time
fname = f'{cache_dir}/snp_annotations.csv'

if os.path.exists(fname):
    print('Cached', fname)
    df_anno = pd.read_csv(fname)
else:
    # retrieve SNP annotations
    df_anno = None
    while df_anno is None:
        try:
            server = pybiomart.Server(host=mart_host)
            dataset = server.marts['ENSEMBL_MART_SNP'].datasets['hsapiens_snp']

            df_anno = dataset.query(
                attributes=['refsnp_id', 'chr_name', 'chrom_start', 'consequence_type_tv'],
                filters={'snp_filter': snps},
                use_attr_names=True)
        except requests.HTTPError:
            # retry if network error occurred
            print('Next try...')
            time.sleep(10)

    # only keep "best" annotation
    df_anno.drop_duplicates(subset='refsnp_id', keep='first', inplace=True)
    
    # mark empty consequence as 'intergenic'
    df_anno.loc[df_anno['consequence_type_tv'].isna(), 'consequence_type_tv'] = 'intergenic_variant'
    
    # set column names
    df_anno.rename(
        columns={
            'refsnp_id': 'snpId', 'consequence_type_tv': 'variant_type',
            'chr_name': 'chromosome', 'chrom_start': 'position'
        }, inplace=True)
    
    # save result
    df_anno.to_csv(fname, index=False)

In [ ]:
df_anno.head()

## Sanity checks

In [ ]:
print('#SNPs in database:', df['snpId'].unique().size, f'({len(snps)})')
print('#annotated SNPs:', df_anno['snpId'].size)
print('#intersection:', len(set(df['snpId'].tolist()) & set(df_anno['snpId'].tolist())))

In [ ]:
assert set(df_anno['snpId'].tolist()) <= set(snps)
not_annotated_snps = set(snps) - set(df_anno['snpId'].tolist())
print(len(not_annotated_snps))

# Infer TAD relations

## Load SNP positions

In [ ]:
df_snppos = df_anno[['snpId', 'chromosome', 'position']].copy()
df_snppos.sample(5)

## Load TAD data

In [ ]:
df_tads = pd.read_table(tad_data_fname)

In [ ]:
df_tads.head()

### TAD statistics

In [ ]:
tad_lengths = df_tads['tad_stop'] - df_tads['tad_start']

In [ ]:
min_, max_ = tad_lengths.min(), tad_lengths.max()
min_ = min_ if min_ > 0 else 1
if min_ == max_:
    min_ -= 10
    max_ += 10

In [ ]:
bins = 10**np.linspace(np.log10(min_), np.log10(max_), 50)
sns.distplot(tad_lengths, kde=False, bins=bins)

plt.xscale('log')
plt.xlabel('TAD length')
plt.ylabel('Count')

plt.tight_layout()
plt.savefig(os.path.join(images_dir, 'tad_length_histogram.pdf'))

## Do work

In [ ]:
def access_range_dict(row, dict_):
    range_dict_ = dict_.get(str(row['chromosome']), None)
    if range_dict_ is None:
        return 'undef'
    
    try:
        return range_dict_[row['position']]
    except KeyError:
        return 'outside'

In [ ]:
tad_anno_20in = parse_tad_annotations('20in', fname=tad_data_fname)
df_snppos['TAD_20in'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_20in), axis=1)

tad_anno_40in = parse_tad_annotations('40in', fname=tad_data_fname)
df_snppos['TAD_40in'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_40in), axis=1)

tad_anno_20out = parse_tad_annotations('20out', fname=tad_data_fname)
df_snppos['TAD_20out'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_20out), axis=1)

tad_anno_40out = parse_tad_annotations('40out', fname=tad_data_fname)
df_snppos['TAD_40out'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_40out), axis=1)

tad_anno_20inout = parse_tad_annotations('20inout', fname=tad_data_fname)
df_snppos['TAD_20inout'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_20inout), axis=1)

tad_anno_40inout = parse_tad_annotations('40inout', fname=tad_data_fname)
df_snppos['TAD_40inout'] = df_snppos.progress_apply(lambda x: access_range_dict(x, tad_anno_40inout), axis=1)

In [ ]:
df_snptads = df_snppos.drop(['chromosome', 'position'], axis=1)
df_snptads.sample(5)

Possible cell values:
* `tad`: SNP is in TAD body (i.e. not in boundary)
* `boundary`: SNP is in TAD boundary
* `undef`: chromosome that SNP is in has no TAD information available
* `outside`: SNP is outside of TAD

# Merge into DisGeNET

In [ ]:
# individual databases
print('DisGeNET only:', df_disgenet.shape)
print('Most recent GWAS-catalog: ', df_gwascat.shape)

In [ ]:
# initial aggregation
df_final = df.copy()
df_final.shape

In [ ]:
# cancer-classification
df_final = df_final.merge(df_iscancer, on='diseaseId')
df_final.shape

In [ ]:
# TAD localization
df_final = df_final.merge(df_snptads, on='snpId')
df_final.shape

In [ ]:
# SNP annotation
df_final = df_final.merge(df_anno, how='left')
df_final.shape

# Subset SNP-sets

In [ ]:
variant_type_counts = (df_final[['snpId', 'variant_type']]
                       .drop_duplicates()['variant_type']
                       .value_counts()
                       .rename('count')
                       .reset_index()
                       .rename(columns={'index': 'variant_type'}))
variant_type_counts

In [ ]:
# TODO: parse ontology (http://www.sequenceontology.org/browser/obob.cgi) properly
exon_type = [
    'missense_variant', 'non_coding_transcript_exon_variant',
    '3_prime_UTR_variant', 'synonymous_variant', '5_prime_UTR_variant'
]
intron_type = ['intron_variant']
intergenic_variant = ['intergenic_variant']

In [ ]:
# remove exonic SNPs
if config['filters']['variant_type'] == 'nonexonic':
    df_final = df_final[~df_final['variant_type'].isin(exon_type)]

# keep only intergenic SNPs
if config['filters']['variant_type'] == 'intergenic':
    df_final = df_final[df_final['variant_type'].isin(intergenic_variant)]

df_final.shape

# Save result

In [ ]:
results_dir = config['output_dirs']['results']
df_final.to_csv(f'{results_dir}/snpdb_enhanced.tsv', sep='\t', index=False)
df_final.sample(5)

# Plot database statistics

## Number of entries per disease

In [ ]:
disease_counts = (df_final['diseaseId']
                  .value_counts()
                  .rename('count')
                  .reset_index()
                  .rename(columns={'index': 'diseaseId'})
                  .sort_values('count')
                  .merge(df_iscancer, on='diseaseId'))

disease_counts.sample(5)

In [ ]:
sns.boxplot(x='is_cancer', y='count', data=disease_counts)

plt.title('#rows associated with single diseases')
plt.yscale('log')

plt.tight_layout()
plt.savefig(f'{images_dir}/disease_count_distribution.pdf')

## Odds ratio distribution

In [ ]:
df_final['odds_ratio'].describe()

In [ ]:
odds_ratio = df_final['odds_ratio'].dropna()
sns.boxplot(odds_ratio[odds_ratio < odds_ratio.quantile(.75)], orient='v')

plt.title('Odds ratios (< 75% quantile) for all diseases')

plt.tight_layout()
plt.savefig(f'{images_dir}/oddsratio_distribution.pdf')

## Variant types

In [ ]:
plt.figure(figsize=(16,8))
sns.barplot(
    x='count', y='variant_type',
    data=variant_type_counts, orient='h', color=sns.color_palette()[0])

plt.title('#variant_type in database')
plt.xscale('log')

plt.tight_layout()
plt.savefig(f'{images_dir}/variant_type_counts.pdf')

## Gene counts

In [ ]:
df_tmp = pd.DataFrame({
    'diseaseId': df_final['diseaseId'],
    'associated_genes': df_final['associated_genes'].str.split(','),
    'gene_count': df_final['associated_genes'].str.split(',').apply(lambda x: 0 if x is None else len([e for e in x if len(e)>0]))
})
df_tmp.sample(5)

In [ ]:
sns.boxplot(y=df_tmp.groupby('diseaseId')['gene_count'].sum())

plt.xlabel('All diseases')
plt.ylabel('#associated genes')

unique_genes = set(g for gs in df_tmp['associated_genes'] if gs is not None for g in gs)
plt.title(f'{len(unique_genes)} unique genes in total')

plt.yscale('log')

plt.tight_layout()
plt.savefig(f'{images_dir}/gene_counts.pdf')